In [3]:
import pandas as pd

# CSV 파일 읽기
money = pd.read_csv(r'C:\Users\m\project\공공데이터_상권분석\data\서울시 상권데이터\서울시_상권분석_서비스(추정매출-상권).csv')  # DataFrame 객체로 로드
human = pd.read_csv(r'C:\Users\m\project\공공데이터_상권분석\data\서울시 상권데이터\서울시_상권분석_서비스(길단위인구-상권).csv')
map_sang = pd.read_csv(r'C:\Users\m\project\공공데이터_상권분석\data\서울시 상권데이터\서울시_상권분석_서비스(영역-상권).csv')

In [4]:
#필요한 컬럼만 선택하기
money = money[['기준_년분기_코드','상권_코드','상권_코드_명','당월_매출_금액']]
human = human[['기준_년분기_코드','상권_코드','총_유동인구_수']]
map_sang = map_sang[['상권_코드','엑스좌표_값','와이좌표_값','자치구_코드_명','행정동_코드_명']]

In [5]:
# 추정매출과 길단위인구 조인 1step
join_1step = pd.merge(money, human, on=['기준_년분기_코드', '상권_코드'], how='inner')

In [14]:
# 1차에서 조인 되지 못한 상권코드
unmatched_from_df1 = money.merge(human, on=['기준_년분기_코드', '상권_코드'], how='left', indicator=True)
unmatched_code = unmatched_from_df1[unmatched_from_df1['_merge'] == 'left_only']

In [15]:
# 1step에서 조인 한것과 영역 데이터 조인
join_final = pd.merge(join_1step, map_sang, on=['상권_코드'], how='inner')

In [17]:
#2차에서 조인 되지 못한 상권코드
unmatched_from_df1 = join_1step.merge(map_sang, on=['상권_코드'], how='left', indicator=True)
unmatched_code2 = unmatched_from_df1[unmatched_from_df1['_merge'] == 'left_only']

In [28]:
# '기준_년분기_코드' 값을 년월로 변환하는 함수 정의
def convert_to_year_month(value):
    year = value // 10  # 앞의 네 자리는 연도
    quarter = value % 10  # 마지막 한 자리는 분기
    month = (quarter - 1) * 3 + 1  # 분기를 기준으로 첫 달 계산
    return f"{year}-{month:02d}"  # YYYY-MM 형식으로 반환
# 변환 적용
join_final['날짜'] = join_final['기준_년분기_코드'].apply(convert_to_year_month)

In [31]:
join_final = join_final[join_final['날짜'] >= '2021-02']

In [32]:
#최종 데이터 저장
join_final.to_csv(r'C:\Users\m\project\공공데이터_상권분석\data\서울시 상권데이터\종합_대시보드데이터.csv', index=False, encoding='utf-8-sig')
unmatched_code.to_csv(r'C:\Users\m\project\공공데이터_상권분석\data\서울시 상권데이터\유동인구없는_상권코드.csv', index=False, encoding='utf-8-sig')